# งานที่ 4: อินเทอร์เฟซการสนทนา - แชทกับ Llama 3 และ Titan Premier LLM

ในโน้ตบุ๊กนี้ คุณจะสร้างแชทบอทโดยใช้ llama3-8b-instruct และ titan-text-premier Foundation Models (FM) ใน Amazon Bedrock

อินเทอร์เฟซการสนทนา เช่น แชทบอทและผู้ช่วยเสมือนสามารถเพิ่มประสบการณ์การใช้งานให้กับลูกค้าของคุณได้ แชทบอทจะใช้การประมวลผลภาษาธรรมชาติ (NLP) และอัลกอริทึมแมชชีนเลิร์นนิง เพื่อทำความเข้าใจและตอบกลับคำถามของผู้ใช้ คุณสามารถใช้แชทบอทในแอปพลิเคชันที่หลากหลาย เช่น การบริการลูกค้า การขาย และอีคอมเมิร์ซ เพื่อให้การตอบกลับผู้ใช้นั้นรวดเร็วและมีประสิทธิภาพ ผู้ใช้สามารถเข้าถึงได้ผ่านช่องทางต่างๆ เช่น เว็บไซต์ แพลตฟอร์มโซเชียลมีเดีย และแอปส่งข้อความ

- **แชทบอท (พื้นฐาน)**, แชทบอท Zero Shot กับโมเดล FM
- **แชทบอทโดยใช้พรอมต์คำสั่ง**, เทมเพลต (LangChain) - แชทบอทที่มีบริบทบางอย่างที่ให้ไว้ในเทมเพลตพรอมต์คำสั่ง
- **แชทบอทกับบุคคล**, แชทบอทที่มีบทบาทตามที่กำหนด เช่น โค้ชอาชีพ และการปฏิสัมพันธ์กับมนุษย์
- **แชทบอทที่ตระหนักถึงบริบท** ส่งผ่านไฟล์ภายนอกโดยสร้างการฝังข้อมูล

## เฟรมเวิร์ก LangChain สำหรับการสร้าง Chatbot กับ Amazon Bedrock

ในอินเทอร์เฟซการสนทนา เช่น แชทบอท การจดจำปฏิสัมพันธ์ก่อนหน้านี้มีความสำคัญอย่างยิ่งทั้งในระดับระยะสั้นและระยะยาว

เฟรมเวิร์ก LangChain มีส่วนประกอบหน่วยความจำในสองรูปแบบ รูปแบบแรกคือ LangChain มียูทิลิตี้ผู้ช่วยสำหรับการจัดการและจัดการข้อความแชทก่อนหน้า สิ่งเหล่านี้ได้รับการออกแบบให้เป็นแบบแยกส่วน รูปแบบที่สองคือ LangChain มีวิธีง่ายๆ ในการรวมยูทิลิตี้เหล่านี้เข้ากับห่วงโซ่ ช่วยให้คุณสามารถกำหนดและโต้ตอบกับนามธรรมประเภทต่างๆ ได้อย่างง่ายดาย ซึ่งทำให้คุณสร้างแชทบอทที่มีประสิทธิภาพได้อย่างง่ายดาย

## การสร้างแชทบอทด้วยบริบท - องค์ประกอบสำคัญ

กระบวนการแรกในการสร้างแชทบอทที่ตระหนักถึงบริบทคือการสร้างการฝังสำหรับบริบท โดยปกติ คุณมีกระบวนการให้ข้อมูลที่ทำงานผ่านโมเดลการฝังของคุณและสร้างการฝังซึ่งจะถูกเก็บไว้ในเวกเตอร์สโตร์ ในโน้ตบุ๊กนี้ ให้คุณใช้รุ่น Titan Embeddings สำหรับสิ่งนี้ กระบวนการที่สองคือการจัดการคำขอของผู้ใช้ การโต้ตอบ การเรียกใช้ และการส่งคืนเอาต์พุต ซึ่งเกี่ยวข้องกับการจัดระเบียบคำขอของผู้ใช้ การโต้ตอบกับโมเดล/ส่วนประกอบที่จำเป็นเพื่อรวบรวมข้อมูล การเรียกแชทบอทเพื่อกำหนดการตอบกลับ จากนั้นก็เป็นการส่งการตอบกลับจากแชทบอทไปยังผู้ใช้

## งานที่ 4.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ

In [ ]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [ ]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## งานที่ 4.2: การใช้ประวัติการแชทจาก LangChain เพื่อเริ่มการสนทนา

ในงานนี้ ให้คุณเปิดใช้งานแชทบอทเพื่อนำบริบทการสนทนาผ่านการโต้ตอบกับผู้ใช้หลายครั้ง การมีหน่วยความจำในการสนทนาเป็นสิ่งสำคัญสำหรับแชทบอทในการจัดบทสนทนาที่มีความหมายและสอดคล้องกันเมื่อเวลาผ่านไป

คุณใช้ความสามารถของหน่วยความจำการสนทนาโดยสร้างบนชั้นเรียน InMemoryChatMessageHistory ของ LangChain อ็อบเจ็กต์นี้จะเก็บการสนทนาระหว่างผู้ใช้และแชทบอท โดยประวัติสามารถใช้งานเอเจนต์แชทบอทเพื่อให้สามารถใช้ประโยชน์จากบริบทจากการสนทนาก่อนหน้านี้ได้

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **หมายเหตุ:** เอาต์พุตของโมเดลไม่สามารถแก้ไขได้

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    model_id="meta.llama3-8b-instruct-v1:0" , 
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

### คำถามใหม่

โมเดลตอบกลับด้วยข้อความเริ่มต้น ตอนนี้ ให้คุณถามคำถามสองสามข้อ

In [ ]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### สร้างจากคำถาม

ตอนนี้ ให้ถามคำถามโดยไม่ต้องพูดถึงคำว่า “สวน” เพื่อดูว่าโมเดลสามารถเข้าใจการสนทนาก่อนหน้านี้ได้หรือไม่

In [ ]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

### เสร็จสิ้นการสนทนานี้

In [ ]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})
print(response)

## งานที่ 4.3: แชทบอทโดยใช้เทมเพลตพรอมต์คำสั่ง (LangChain)

ในงานนี้ ให้คุณใช้ PromptTemplate เริ่มต้นที่รับผิดชอบในการสร้างอินพุตนี้ LangChain มีคลาสและฟังก์ชั่นหลายอย่างเพื่อให้การสร้างและการทำงานกับพรอมต์คำสั่งทำได้ง่าย

In [ ]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [ ]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        response = self.qa.invoke({"input": prompt})
                        result=response['answer']
                    else:
                        instructions = [{"role": "user", "content": prompt}]
                        #result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                        result = self.qa.invoke({"input": format_instructions(instructions)})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

ถัดไป ให้เริ่มแชท

In [ ]:
# start chat
history = InMemoryChatMessageHistory() #reset chat history
chat = ChatUX(wrapped_chain)
chat.start_chat()

In [ ]:
print(history)

## งานที่ 4.4: แชทบอทที่มีบุคลิกภาพ

ในงานนี้ ผู้ช่วยปัญญาประดิษฐ์ (AI) จะมีบทบาทเป็นโค้ชอาชีพ คุณสามารถแจ้งแชทบอทเกี่ยวกับบุคลิกภาพ (หรือบทบาท) โดยใช้ข้อความระบบ ใช้ประโยชน์จากการสอนเรื่อง InMemoryChatMessageHistory ต่อไป เพื่อรักษาบริบทการสนทนา

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", " You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory() # reset history

chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="career_chat_history",
)

response=wrapped_chain.invoke({"input": "What are the career options in AI?"})
print(response)

In [ ]:
response=wrapped_chain.invoke({"input": "How to fix my car?"})
print(response)

In [ ]:
print(history)

ตอนนี้ ให้ถามคำถามที่ไม่ได้อยู่ในความพิเศษของบุคคลนี้ โมเดลไม่ควรตอบคำถามนั้นและควรให้เหตุผลสำหรับสิ่งนั้น

## งาน 4.5 แชทบอทพร้อมบริบท

ในงานนี้ ให้คุณขอให้แชทบอทตอบคำถามตามบริบทที่ส่งต่อไป คุณใช้ไฟล์ CSV และใช้โมเดล Titan Embeddings เพื่อสร้างเวกเตอร์ที่เป็นตัวแทนบริบทนั้น เวกเตอร์นี้ถูกเก็บไว้ใน Facebook AI Similarity Search (FAISS) เมื่อแชทบอทถูกถามคำถาม คุณจะส่งเวกเตอร์นี้กลับไปยังแชทบอทและขอให้ดึงคำตอบโดยใช้เวกเตอร์

### โมเดลการฝัง Titan

การฝังจะแสดงคำ วลี หรือรายการที่ไม่ต่อเนื่องอื่นๆ เป็นเวกเตอร์ในพื้นที่เวกเตอร์ต่อเนื่อง สิ่งนี้ช่วยให้โมเดลแมชชีนเลิร์นนิงสามารถดำเนินการทางคณิตศาสตร์ในการแสดงเหล่านี้และจับภาพความสัมพันธ์เชิงความหมายเหล่านี้

คุณใช้การฝังสำหรับการสร้างการดึงข้อมูลเพิ่มเติม (RAG) [ความสามารถในการค้นหาเอกสาร] (https://labelbox.com/blog/how-vector-similarity-search-works/)

In [ ]:
# model configuration
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS เป็น VectorStore

ในการใช้การฝังสำหรับการค้นหา คุณต้องมีร้านค้าที่สามารถทำการค้นหาความคล้ายคลึงกันของเวกเตอร์ได้อย่างมีประสิทธิภาพ ในโน้ตบุ๊กนี้ ให้คุณใช้ FAISS ซึ่งเป็นที่เก็บในหน่วยความจำ ในการจัดเก็บเวกเตอร์อย่างถาวร คุณสามารถใช้ฐานความรู้สำหรับ Amazon Bedrock, pgVector, Pinecone, Weaviate หรือ Chroma ได้

API ของ LangChain VectorStore อยู่ [ที่นี่](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

In [ ]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("../rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

### เรียกใช้การทดสอบรหัสต่ำอย่างรวดเร็ว 

คุณสามารถใช้คลาส Wrapper ที่จัดทำโดย LangChain เพื่อสืบค้นที่เก็บฐานข้อมูลเวกเตอร์และส่งคืนเอกสารที่เกี่ยวข้อง สิ่งนี้จะเรียกใช้ QA Chain ที่มีค่าเริ่มต้นทั้งหมด

In [ ]:
chat_llm=ChatBedrock(
    model_id="amazon.titan-text-premier-v1:0" , 
    client=bedrock_client)
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=chat_llm))

### แอปพลิเคชันแชทบอท

สำหรับแชทบอท คุณต้องมีการจัดการบริบท ประวัติ ที่จัดเก็บเวกเตอร์ และส่วนประกอบอื่นๆ อีกมากมาย คุณเริ่มต้นด้วยการสร้างห่วงโซ่ Recrieval Augmented Generation (RAG) ที่รองรับบริบท

สิ่งนี้ใช้ฟังก์ชัน **create_stuff_documents_chain** และ**create_retrieval_chain**

### พารามิเตอร์และฟังก์ชั่นที่ใช้สำหรับ RAG

- **Retriever:** คุณใช้ `VectorStoreRetriever` ซึ่งได้รับการสนับสนุนโดย `VectorStore` ในการดึงข้อความ มีสองประเภทการค้นหาให้เลือก คือ `"similarity"` หรือ `"mmr"` โดย `search_type="similarity"` จะใช้การค้นหาความคล้ายคลึงกันในอ็อบเจ็กต์รีทริเวอร์ ซึ่งจะเลือกเวกเตอร์ชิ้นข้อความที่คล้ายกับเวกเตอร์คำถามมากที่สุด

- **create_stuff_documents_chain** จะระบุว่าบริบทที่ดึงข้อมูลจะถูกป้อนเข้าสู่พรอมต์คำสั่งและ LLM อย่างไร เอกสารที่ดึงมาจะเป็น “stuffed” ตามบริบทโดยไม่มีการสรุปหรือการประมวลผลอื่นๆ ลงในพรอมต์คำสั่ง

- **create_retrieval_chain** จะเพิ่มขั้นตอนการดึงข้อมูลและเผยแพร่บริบทที่ดึงข้อมูลผ่านห่วงโซ่โดยให้พร้อมกับคำตอบสุดท้าย 

หากคำถามที่ถามอยู่นอกขอบเขตของบริบทโมเดลจะตอบว่าไม่ทราบคำตอบ

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever=vectorstore_faiss_aws.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is sagemaker?"})
print(response) # shows the document chunks consulted to come up with the answer

ถัดไป ให้เริ่มแชท

In [ ]:
chat = ChatUX(rag_chain, retrievalChain=True)
chat.start_chat()  # Only answers will be shown here, and not the citations


คุณได้ใช้ Titan LLM เพื่อสร้างอินเทอร์เฟซการสนทนาด้วยรูปแบบต่อไปนี้:

- แชทบอท (พื้นฐาน - ไม่มีบริบท)
- แชทบอทโดยใช้เทมเพลตพรอมต์คำสั่ง (Langchain)
- แชทบอทกับบุคคล
- แชทบอทที่มีบริบท

### ลองด้วยตัวเอง

- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้และไปต่อยัง **งานที่ 5**